In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

#helper function for avg_slope intercept
def make_coordinates(image,line_params):
    slope,intercept = line_params
    #start at the bottom, go upwards 3/5(~420px)
    y1 = image.shape[0] 
    y2 = int(y1*(3/5)) 
    x1 = int((y1-intercept)/slope)
    x2 = int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])

# average hough lines into smoother ones
def avg_slope_intercept(image,lines):
    left_fit = []
    right_fit = []
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2 = line.reshape(4)
            #create polynomial out of 2 points
            params = np.polyfit((x1,x2),(y1,y2),1) 
            slope = params[0]
            intercept = params[1]
            if slope < 0:
                left_fit.append((slope,intercept))
            else: 
                right_fit.append((slope,intercept))
        left_fit_avg = np.average(left_fit,axis=0)
        right_fit_avg = np.average(right_fit,axis=0)
        left_line = make_coordinates(image,left_fit_avg)
        right_line = make_coordinates(image,right_fit_avg)
        return np.array([left_line,right_line])
    
# detect lines (high filter)
def canny(image):
#     blur = cv.boxFilter(image,-1,(5,5))
    return cv.Canny(image,100,150)

# create region of interest
def roi(image):
    height = image.shape[0] 
    # array with 1 polygon (3 points)
    poly = np.array([[(200,height),(1100,height),(550,250)]])
    # create mask (init as zeros)
    mask = np.zeros_like(image)
    # add a white triangle (road section)
    cv.fillPoly(mask,poly,255)
    # AND operation (crop the road section)
    crop = cv.bitwise_and(image,mask)
    return crop
  
# draw the detected lines
def display_lines(image,lines):
    line_image = np.zeros_like(image)
    if lines is not None:
        for x1,y1,x2,y2 in lines:
            cv.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image
   
    
    
    
# test image
# img = cv.imread('test_image.jpg')

# load video    
cap = cv.VideoCapture('test2.mp4')

while (cap.isOpened()):
    _,img = cap.read()
    gray_img = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

    canny_img = canny(gray_img) # detect lines

    cropped_img = roi(canny_img) # crop mask area

    # create lines
    lines = cv.HoughLinesP(cropped_img,2,np.pi/180,100,np.array([]),
                           minLineLength=40,maxLineGap=5) 

    # smooth lines (combine them into single lines)
    avg_lines = avg_slope_intercept(img,lines)

    # draw the lines
    line_image = display_lines(img,avg_lines)

    # combine images (src1,w1,src2,w2,gamma)
    final = cv.addWeighted(img,0.8,line_image,1,1)
    

    cv.imshow("res",final)
    if cv.waitKey(1) & 0xFF == ord('q'): 
        break
        
cap.release()
cv.destroyAllWindows()

